# Limit Opinion Consensus Check for Symmetric Case, Circle Network

#### Unified Model with Multiple Strategic Agents
#### Strategic Agent-Specific Message Weights

James Yu, 20 January 2023, revised 24 February 2023, updated 31 May 2024

In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def M(K, B, R, L, delta):
    """Computes M_{t-1} given B_l \forall l, K_t^l \forall l, 
        R_l \forall l, number of strategic agents L, and delta."""
    # handle the generic structure first, with the correct pairings:
    base = [[B[l_prime].T @ K[l_prime] @ B[l] for l in range(L)] for l_prime in range(L)]
    # then change the diagonals to construct M_{t-1}:
    for l in range(L): base[l][l] = B[l].T @ K[l] @ B[l] + R[l]/delta
    return np.block(base)

def H(B, K, A, L):
    """Computes H_{t-1} given B_l \forall l, K_t^l \forall l, 
        A, and number of strategic agents L."""
    return np.concatenate(tuple(B[l].T @ K[l] @ A for l in range(L)), axis = 0)

def C_l(A, B, K, k, h, L, c, x, n):
    """Computes C_{t-1}^h (displayed as C_{t-1}^l) given A, B_l \forall l, K_t^l \forall l, 
        k_t^l \forall l, a specific naive agent h, number of strategic agents L, 
        c_l \forall l, x_l \forall l, and number of naive agents n"""
    return np.concatenate(tuple(B[l].T @ K[l] @ A @ ((x[h] - x[l]) * np.ones((n, 1))) 
                           + B[l].T @ K[l] @ c[l] 
                           + 0.5 * B[l].T @ k[l].T for l in range(L)), axis = 0)

def E(M_, H_):
    """Computes the generic E_{t-1} given M_{t-1} and H_{t-1}."""
    return np.linalg.inv(M_) @ H_

def F(M_, C_l_, l, n):
    """Computes F_{t-1}^l given M_{t-1}, C_{t-1}^l, 
       specific naive agent l and number of naive agents n."""
    return (np.linalg.inv(M_) @ C_l_)[l*n:(l+1)*n, :] # e.g. l = 0 gives ln = 0, l = 1 gives ln = n, etc

def G(A, B, E_, L, n):
    """Computes the generic G_{t-1} given A, B_l \forall l, 
        E_{t-1}, number of strategic agents L, and number of naive agents n."""
    return A - sum([B[l] @ E_[l*n:(l+1)*n, :] for l in range(L)])
    
def g_l(B, E_, h, x, F_, L, n, c):
    """Computes g_{t-1}^l given B_l \forall l, E_{t-1}^l, 
        a particular naive agent h, x_l \forall l, F_{t-1}^l \forall l, 
        number of strategic agents L, number of naive agents n, and c_h."""
    return - sum([B[l] @ (E_[l*n:(l+1)*n, :] @ ((x[h] - x[l]) * np.ones((n, 1))) + F_[l]) for l in range(L)]) + c[h]

In [3]:
def K_t_minus_1(Q, K, E_, R, G_, L, delta, n):
    return [Q[l] + E_[l*n:(l+1)*n, :].T @ R[l] @ E_[l*n:(l+1)*n, :] 
            + delta * G_.T @ K[l] @ G_ for l in range(L)]

def k_t_minus_1(K, k, G_, g, E_, F_, R, L, delta, n):
    return [2*delta* g[l].T @ K[l] @ G_ + delta * k[l] @ G_ 
            + 2 * F_[l].T @ R[l] @ E_[l*n:(l+1)*n, :] for l in range(L)]

def kappa_t_minus_1(K, k, kappa, g_, F_, R, L, delta):            
    return [-delta * (g_[l].T @ K[l] @ g_[l] + k[l] @ g_[l] - kappa[l]) 
            - (F_[l].T @ R[l] @ F_[l]) for l in range(L)]

In [4]:
def should_terminate(bundles, eps):
    return all([np.allclose(b[0], b[1], rtol = eps, atol = eps) for b in bundles])

def solve(K_t, k_t, kappa_t, A, B, delta, n, L, Q, R, x, c, tol):
    historical_K = [K_t]
    historical_k = [k_t]
    historical_kappa = [kappa_t]
    eps = np.sqrt(np.finfo(np.float64).eps)
    while True:
        M_ = M(K_t, B, R, L, delta)
        H_ = H(B, K_t, A, L)
        E_ = E(M_, H_)
        G_ = G(A, B, E_, L, n)
        K_new = K_t_minus_1(Q, K_t, E_, R, G_, L, delta, n)
        F_ = [F(M_, C_l(A, B, K_t, k_t, l, L, c, x, n), l, n) for l in range(L)]
        g = [g_l(B, E_, h, x, F_, L, n, c) for h in range(L)]
        k_new = k_t_minus_1(K_t, k_t, G_, g, E_, F_, R, L, delta, n)
        kappa_new = kappa_t_minus_1(K_t, k_t, kappa_t, g, F_, R, L, delta)
        historical_K.insert(0, K_new)
        historical_k.insert(0, k_new)
        historical_kappa.insert(0, kappa_new)
        if should_terminate([(K_t, K_new), (k_t, k_new), (kappa_t, kappa_new)], eps):
            return historical_K[0], historical_k[0], historical_kappa[0]
        K_t = K_new
        k_t = k_new
        kappa_t = kappa_new

In [5]:
def optimal(X_init, K_ss, k_ss, A, B, delta, n, L, Q, R, x, c, eps):
    X_t = [a.copy() for a in X_init]
    xs = defaultdict(list)
    for l in range(L):
        xs[l].append(X_t[l])
        
    rs = defaultdict(list)
    payoffs = defaultdict(list)
    payoff = defaultdict(int)
    
    M_ = M(K_ss, B, R, L, delta)
    H_ = H(B, K_ss, A, L)
    E_ = E(M_, H_)
    G_ = G(A, B, E_, L, n)
    F_ = [F(M_, C_l(A, B, K_ss, k_ss, l, L, c, x, n), l, n) for l in range(L)]
    g = [g_l(B, E_, h, x, F_, L, n, c) for h in range(L)]
    
    i = 0
    while True:
        for l in range(L):
            # TODO: special code for finite horizon needs a terminal time T term
            Y_new = -1 * E_[l*n:(l+1)*n, :] @ X_t[l] - F(M_, C_l(A, B, K_ss, k_ss, l, L, c, x, n), l, n)
            rs[l].append(Y_new)
            payoff[l] += (-1 * delta**i * (X_t[l].T @ Q[l] @ X_t[l])).item() + (-1 * delta**i * (Y_new.T @ R[l] @ Y_new)).item()
            payoffs[l].append(payoff[l])
            X_new = G_ @ X_t[l] + g[l]
            xs[l].append(X_new)
            if l == L - 1 and np.allclose(X_t[l], X_new, rtol = eps, atol = eps):
                return xs, rs, payoffs
            X_t[l] = X_new 
        i += 1
        
    return xs, rs, payoffs

In [6]:
def run_simulation(b1, b2, a1, a2, delta = 0.9, c = 200, X_0_1 = np.array([[10.0, -5.0, 5.0, 1.0]], ndmin = 2).T, symmetric = True, print_ = True, tol = np.sqrt(np.finfo(np.float64).eps)):
    if symmetric:
        A = np.array([
            [1/3, 1/3, 0, 1/3],
            [1/3, 1/3, 1/3, 0],
            [0, 1/3, 1/3, 1/3],
            [1/3, 0, 1/3, 1/3],
        ])
        if print_: print("SYMMETRIC NETWORK")
    else:
        raise Exception("asymmetric network not allowed")
    X_0 = [X_0_1 - b1, X_0_1 - b2]
    n = 4 # number of naive agents
    L = 2 # number of strategic agents
    Q = [np.identity(n), np.identity(n)] # objective function for messages is just X'IX = X'X
    R = [c * np.identity(n), c * np.identity(n)] # message cost R = cI_n for some c under the new notation
    B = [a1 * np.identity(n), a2 * np.identity(n)] # B^l = a_l I_n
    x = [b1, b2] # agendas
    r = [0, 0] # message cost minimality is centered around zero
    c_base = sum([B[l] @ (r[l] * np.ones((n, 1))) for l in range(L)])
    c = [c_base + (A - np.identity(n)) @ (x[l] * np.ones((n, 1))) for l in range(L)] # normalization vector
    
    K_ss, k_ss, kappa_ss = solve(Q, [np.zeros((1, n)), np.zeros((1, n))], [0, 0], A, B, delta, n, L, Q, R, x, c, tol)
    xs, rs, payoffs = optimal(X_0, K_ss, k_ss, A, B, delta, n, L, Q, R, x, c, tol)
    if print_:
        print("Steady-State Opinions:")
        print(xs[0][-1] + b1)
        print("Steady-State Messages:")
        print("1", rs[0][-1])
        print("2", rs[1][-1])
        print("Total Payoffs:")
        print("1", payoffs[0][-1])
        print("2", payoffs[1][-1])
        print("Average of agendas:")
        print((b1+b2)/2)
        print()
        print("K^* (1, 2):")
        print(K_ss[0])
        print(K_ss[1])
        print()
        print("k^* (1, 2):")
        print(k_ss[0])
        print(k_ss[1])
        print()
        print("kappa^* (1, 2):")
        print(kappa_ss[0])
        print(kappa_ss[1])
    if print_ == False:
        return xs[0][-1] + b1

In [7]:
run_simulation(0, 0, 1, 1) #b1, b2 agendas, followed by alpha_1, alpha_2 weights

SYMMETRIC NETWORK
Steady-State Opinions:
[[2.63506216e-07]
 [2.63506216e-07]
 [2.63506216e-07]
 [2.63506216e-07]]
Steady-State Messages:
1 [[-7.07410222e-09]
 [-7.07410222e-09]
 [-7.07410222e-09]
 [-7.07410222e-09]]
2 [[-7.07410222e-09]
 [-7.07410222e-09]
 [-7.07410222e-09]
 [-7.07410222e-09]]
Total Payoffs:
1 -314.41187488059586
2 -314.4118748805958
Average of agendas:
0.0

K^* (1, 2):
[[2.32341466 1.21412507 1.21412507 1.21412507]
 [1.21412507 2.32341466 1.21412507 1.21412507]
 [1.21412507 1.21412507 2.32341466 1.21412507]
 [1.21412507 1.21412507 1.21412507 2.32341466]]
[[2.32341466 1.21412507 1.21412507 1.21412507]
 [1.21412507 2.32341466 1.21412507 1.21412507]
 [1.21412507 1.21412507 2.32341466 1.21412507]
 [1.21412507 1.21412507 1.21412507 2.32341466]]

k^* (1, 2):
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]

kappa^* (1, 2):
[[-0.]]
[[-0.]]


In [8]:
run_simulation(10000, 1000000, 5000, 600)

SYMMETRIC NETWORK
Steady-State Opinions:
[[11625.8763875 ]
 [11625.8763875 ]
 [11625.87638774]
 [11625.87638726]]
Steady-State Messages:
1 [[-324379.99357609]
 [-324379.99357608]
 [-324379.99357597]
 [-324379.993576  ]]
2 [[2703166.61313405]
 [2703166.61313401]
 [2703166.61313305]
 [2703166.61313336]]
Total Payoffs:
1 -228121348837118.94
2 -1.5852496963364466e+16
Average of agendas:
505000.0

K^* (1, 2):
[[1.00000259e+00 1.72764834e-06 1.72764834e-06 1.72764834e-06]
 [1.72764834e-06 1.00000259e+00 1.72764834e-06 1.72764834e-06]
 [1.72764834e-06 1.72764834e-06 1.00000259e+00 1.72764834e-06]
 [1.72764834e-06 1.72764834e-06 1.72764834e-06 1.00000259e+00]]
[[1.00000004e+00 2.48928478e-08 2.48928478e-08 2.48928478e-08]
 [2.48928478e-08 1.00000004e+00 2.48928478e-08 2.48928478e-08]
 [2.48928478e-08 2.48928478e-08 1.00000004e+00 2.48928478e-08]
 [2.48928478e-08 2.48928478e-08 2.48928478e-08 1.00000004e+00]]

k^* (1, 2):
[[25581.99915086 25581.99915086 25581.99915086 25581.99915086]]
[[-25597.

In [9]:
run_simulation(0, 1, -0.5, 0.6)

SYMMETRIC NETWORK
Steady-State Opinions:
[[0.5982444]
 [0.5982444]
 [0.5982444]
 [0.5982444]]
Steady-State Messages:
1 [[0.01209458]
 [0.01209458]
 [0.01209458]
 [0.01209458]]
2 [[0.01007878]
 [0.01007878]
 [0.01007878]
 [0.01007878]]
Total Payoffs:
1 -384.6021600279009
2 -223.65118655571663
Average of agendas:
0.5

K^* (1, 2):
[[2.75631924 1.64580379 1.64580379 1.64580379]
 [1.64580379 2.75631924 1.64580379 1.64580379]
 [1.64580379 1.64580379 2.75631924 1.64580379]
 [1.64580379 1.64580379 1.64580379 2.75631924]]
[[2.81101084 1.70042799 1.70042799 1.70042799]
 [1.70042799 2.81101084 1.70042799 1.70042799]
 [1.70042799 1.70042799 2.81101084 1.70042799]
 [1.70042799 1.70042799 1.70042799 2.81101084]]

k^* (1, 2):
[[1.54527584 1.54527584 1.54527584 1.54527584]]
[[-1.10814743 -1.10814743 -1.10814743 -1.10814743]]

kappa^* (1, 2):
[[-0.77403412]]
[[-0.37972815]]


In [10]:
run_simulation(0, 5, 0.5, 0.6, c = 0.001)

SYMMETRIC NETWORK
Steady-State Opinions:
[[3.29858496]
 [3.29858496]
 [3.29858496]
 [3.29858496]]
Steady-State Messages:
1 [[-2355.20991324]
 [-2355.20991324]
 [-2355.20991324]
 [-2355.20991324]]
2 [[1962.6749277 ]
 [1962.67492771]
 [1962.6749277 ]
 [1962.6749277 ]]
Total Payoffs:
1 -76553.38948242347
2 -53167.12953539577
Average of agendas:
2.5

K^* (1, 2):
[[1.00022407e+00 1.49363747e-04 1.49363747e-04 1.49363747e-04]
 [1.49363747e-04 1.00022407e+00 1.49363747e-04 1.49363747e-04]
 [1.49363747e-04 1.49363747e-04 1.00022407e+00 1.49363747e-04]
 [1.49363747e-04 1.49363747e-04 1.49363747e-04 1.00022407e+00]]
[[1.00032237e+00 2.14933746e-04 2.14933746e-04 2.14933746e-04]
 [2.14933746e-04 1.00032237e+00 2.14933746e-04 2.14933746e-04]
 [2.14933746e-04 2.14933746e-04 1.00032237e+00 2.14933746e-04]
 [2.14933746e-04 2.14933746e-04 2.14933746e-04 1.00032237e+00]]

k^* (1, 2):
[[3.86599532 3.86599532 3.86599532 3.86599532]]
[[-3.86304519 -3.86304519 -3.86304519 -3.86304519]]

kappa^* (1, 2):
[[-

In [11]:
run_simulation(0, 50000000, 0.5, 0.6, c = 0.001)

SYMMETRIC NETWORK
Steady-State Opinions:
[[32985849.61915468]
 [32985849.61915351]
 [32985849.61915379]
 [32985849.61915847]]
Steady-State Messages:
1 [[-2.35520991e+10]
 [-2.35520991e+10]
 [-2.35520991e+10]
 [-2.35520991e+10]]
2 [[1.96267493e+10]
 [1.96267493e+10]
 [1.96267493e+10]
 [1.96267493e+10]]
Total Payoffs:
1 -7.635983418427891e+18
2 -5.316868717598698e+18
Average of agendas:
25000000.0

K^* (1, 2):
[[1.00022407e+00 1.49363747e-04 1.49363747e-04 1.49363747e-04]
 [1.49363747e-04 1.00022407e+00 1.49363747e-04 1.49363747e-04]
 [1.49363747e-04 1.49363747e-04 1.00022407e+00 1.49363747e-04]
 [1.49363747e-04 1.49363747e-04 1.49363747e-04 1.00022407e+00]]
[[1.00032237e+00 2.14933746e-04 2.14933746e-04 2.14933746e-04]
 [2.14933746e-04 1.00032237e+00 2.14933746e-04 2.14933746e-04]
 [2.14933746e-04 2.14933746e-04 1.00032237e+00 2.14933746e-04]
 [2.14933746e-04 2.14933746e-04 2.14933746e-04 1.00032237e+00]]

k^* (1, 2):
[[38659953.18001004 38659953.18000874 38659953.18001001 38659953.1800

In [12]:
run_simulation(0, 50000000, 0.5, 0.6)

SYMMETRIC NETWORK
Steady-State Opinions:
[[29912158.04595322]
 [29912158.04595322]
 [29912158.04595322]
 [29912158.04595322]]
Steady-State Messages:
1 [[-604727.95910022]
 [-604727.95910022]
 [-604727.95910022]
 [-604727.95910022]]
2 [[503940.45639053]
 [503940.45639053]
 [503940.45639053]
 [503940.45639053]]
Total Payoffs:
1 -1935086711991185.8
2 -9.115373379748675e+16
Average of agendas:
25000000.0

K^* (1, 2):
[[2.75631924 1.64580379 1.64580379 1.64580379]
 [1.64580379 2.75631924 1.64580379 1.64580379]
 [1.64580379 1.64580379 2.75631924 1.64580379]
 [1.64580379 1.64580379 1.64580379 2.75631924]]
[[2.81101084 1.70042799 1.70042799 1.70042799]
 [1.70042799 2.81101084 1.70042799 1.70042799]
 [1.70042799 1.70042799 2.81101084 1.70042799]
 [1.70042799 1.70042799 1.70042799 2.81101084]]

k^* (1, 2):
[[77263792.3826329  77263792.38263288 77263792.3826329  77263792.3826329 ]]
[[-55407371.95950881 -55407371.95950881 -55407371.95950881
  -55407371.9595088 ]]

kappa^* (1, 2):
[[-1.93508562e+15

In [13]:
run_simulation(-20, 5, 0.5, 0.6, c = 0.001, delta = 0.9)

SYMMETRIC NETWORK
Steady-State Opinions:
[[-3.50707519]
 [-3.50707519]
 [-3.50707519]
 [-3.50707519]]
Steady-State Messages:
1 [[-11776.04956624]
 [-11776.04956624]
 [-11776.04956624]
 [-11776.04956624]]
2 [[9813.37463847]
 [9813.37463847]
 [9813.37463847]
 [9813.37463847]]
Total Payoffs:
1 -1912947.2829364175
2 -1325098.1088399256
Average of agendas:
-7.5

K^* (1, 2):
[[1.00022407e+00 1.49363747e-04 1.49363747e-04 1.49363747e-04]
 [1.49363747e-04 1.00022407e+00 1.49363747e-04 1.49363747e-04]
 [1.49363747e-04 1.49363747e-04 1.00022407e+00 1.49363747e-04]
 [1.49363747e-04 1.49363747e-04 1.49363747e-04 1.00022407e+00]]
[[1.00032237e+00 2.14933746e-04 2.14933746e-04 2.14933746e-04]
 [2.14933746e-04 1.00032237e+00 2.14933746e-04 2.14933746e-04]
 [2.14933746e-04 2.14933746e-04 1.00032237e+00 2.14933746e-04]
 [2.14933746e-04 2.14933746e-04 2.14933746e-04 1.00032237e+00]]

k^* (1, 2):
[[19.32997659 19.32997659 19.32997659 19.32997659]]
[[-19.31522594 -19.31522594 -19.31522594 -19.31522594]]



In [14]:
run_simulation(10, 100, 0.5, 0.50002, c = 0.001, delta = 0.9)

SYMMETRIC NETWORK
Steady-State Opinions:
[[55.00327353]
 [55.00327353]
 [55.00327353]
 [55.00327353]]
Steady-State Messages:
1 [[-36886.50276215]
 [-36886.50276215]
 [-36886.50276215]
 [-36886.50276215]]
2 [[36885.02736218]
 [36885.02736219]
 [36885.02736219]
 [36885.02736218]]
Total Payoffs:
1 -18721255.014074966
2 -18788278.14801305
Average of agendas:
55.0

K^* (1, 2):
[[1.00033331e+00 2.22203375e-04 2.22203375e-04 2.22203375e-04]
 [2.22203375e-04 1.00033331e+00 2.22203375e-04 2.22203375e-04]
 [2.22203375e-04 2.22203375e-04 1.00033331e+00 2.22203375e-04]
 [2.22203375e-04 2.22203375e-04 2.22203375e-04 1.00033331e+00]]
[[1.00033333e+00 2.22221112e-04 2.22221112e-04 2.22221112e-04]
 [2.22221112e-04 1.00033333e+00 2.22221112e-04 2.22221112e-04]
 [2.22221112e-04 2.22221112e-04 1.00033333e+00 2.22221112e-04]
 [2.22221112e-04 2.22221112e-04 2.22221112e-04 1.00033333e+00]]

k^* (1, 2):
[[73.8434663 73.8434663 73.8434663 73.8434663]]
[[-73.84345193 -73.84345193 -73.84345193 -73.84345193]]

k

In [15]:
run_simulation(-20, 5, 0.5, 0.6, c = 1.0, delta = 0.9)

SYMMETRIC NETWORK
Steady-State Opinions:
[[-3.5113144]
 [-3.5113144]
 [-3.5113144]
 [-3.5113144]]
Steady-State Messages:
1 [[-20.85698495]
 [-20.85698495]
 [-20.85698495]
 [-20.85698495]]
2 [[17.38082059]
 [17.38082059]
 [17.38082059]
 [17.38082059]]
Total Payoffs:
1 -29483.367800335604
2 -12970.722666582456
Average of agendas:
-7.5

K^* (1, 2):
[[1.16992238 0.11774181 0.11774181 0.11774181]
 [0.11774181 1.16992238 0.11774181 0.11774181]
 [0.11774181 0.11774181 1.16992238 0.11774181]
 [0.11774181 0.11774181 0.11774181 1.16992238]]
[[1.20395441 0.14706073 0.14706073 0.14706073]
 [0.14706073 1.20395441 0.14706073 0.14706073]
 [0.14706073 0.14706073 1.20395441 0.14706073]
 [0.14706073 0.14706073 0.14706073 1.20395441]]

k^* (1, 2):
[[42.46830067 42.46830067 42.46830067 42.46830067]]
[[-36.36885964 -36.36885964 -36.36885964 -36.36885964]]

kappa^* (1, 2):
[[-23818.19997568]]
[[-13266.5162143]]
